In [2]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [3]:
#路径

path='C:\Users\Wvv\Datacastle_subsidy'

#导入dorm_train数据

dorm_test_name=['id','dorm_date','out_or_in']
dorm_test=pd.read_table(path+'/final_test/dorm_final_test.txt',sep=',',header=None,names=dorm_test_name)

#删除重复行并浏览前几行数据
dorm_test=dorm_test.drop_duplicates()
dorm_test.head()

,id,dorm_date,out_or_in
0,28358,2014/01/21 05:58:02,1
1,14167,2014/01/21 07:08:13,1
2,14167,2014/01/21 07:14:19,0
3,8280,2014/01/21 08:33:22,0
4,1511,2014/01/21 09:30:28,1


In [3]:
#路径

path='C:\Users\Wvv\Datacastle_subsidy'

#导入dorm_train数据

dorm_test_name=['id','dorm_date','out_or_in']
dorm_test=pd.read_table(path+'/test/dorm_test.txt',sep=',',header=None,names=dorm_test_name)

#删除重复行并浏览前几行数据
dorm_test=dorm_test.drop_duplicates()
dorm_test.head()

,id,dorm_date,out_or_in
0,19326,2014/01/21 06:12:13,0
1,15190,2014/01/21 06:49:12,1
2,18610,2014/01/21 06:51:00,0
3,462,2014/01/21 07:40:51,1
4,18610,2014/01/21 07:51:06,1


In [4]:
dorm_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2139942 entries, 0 to 2140486
Data columns (total 3 columns):
id           int64
dorm_date    object
out_or_in    int64
dtypes: int64(2), object(1)
memory usage: 65.3+ MB


In [4]:
#定义time转化函数

from datetime import datetime
def time_transform(x):
    return datetime.strptime(x,'%Y/%m/%d %H:%M:%S')

In [5]:
#转换字符串型的time成时间戳格式

dorm_test['dorm_date']=dorm_test['dorm_date'].apply(time_transform)

In [6]:
#时间格式转换

dorm_test['date']=dorm_test['dorm_date'].apply(lambda x:x.strftime('%m-%d-%Y'))
dorm_test['year']=dorm_test['dorm_date'].apply(lambda x:x.strftime('%Y'))
dorm_test['month']=dorm_test['dorm_date'].apply(lambda x:x.strftime('%m'))
dorm_test['hour']=dorm_test['dorm_date'].apply(lambda x:x.strftime('%H'))
dorm_test['weekday']=dorm_test['dorm_date'].apply(lambda x:x.strftime('%A'))

In [7]:
#上面数据处理花时很多，先保存处理后的结果

#dorm_test.to_csv('dorm_input/dorm_test_time.csv',index=False)

In [8]:
#上面数据处理花时很多，先保存处理后的结果

dorm_test.to_csv('dorm_input/dorm_test_time_2.csv',index=False)

In [9]:
#对hour进行one_hot编码得到hour新表

hour_dummies=pd.get_dummies(dorm_test['hour'])
hour_dummies.columns=['dorm_hour'+x for x in list(hour_dummies.columns)]

#连接两表

dorm_test=dorm_test.join(hour_dummies)

In [10]:
#hour 

#分为4个时段

a=['01','02','03','04','05']
b=['06','07','08','09','10','11']
c=['12','13','14','15','16','17']
d=['18','19','20']
#e=['21','22','23','00']
def get_hour_transform(hour):
    if hour in a:
        return 'a'
    elif hour in b:
        return 'b'
    elif hour in c:
        return 'c'
    elif hour in d:
        return 'd'
    else:
        return 'e'
    
#运用函数
dorm_test['hour']=dorm_test['hour'].apply(get_hour_transform)

#实现one-hot编码
hour_dummies1=pd.get_dummies(dorm_test['hour'])
hour_dummies1.columns=['dorm_hour_01_05','dorm_hour_morning','dorm_hour_afternoon','dorm_hour_evening','dorm_hour_midnight']

#连接两表
dorm_test=dorm_test.join(hour_dummies1)

In [11]:
#weekday

#分为周末与非周末
a=['Monday','Tuesday','Wednesday','Thursday','Friday']
#b=['Saturday','Sunday']
def week_transform(weekday):
    if weekday in a:
        return 'wd'
    else:
        return 'vd'
    
dorm_test['weekday']=dorm_test['weekday'].apply(week_transform)

#对weekday进行one_hot编码得到weekday新表

weekday_dummies = pd.get_dummies(dorm_test['weekday'])
weekday_dummies.columns=['dorm_resting_day','dorm_working_day']
dorm_test=dorm_test.join(weekday_dummies)

In [12]:
#month

#分为在校与假期
#dorm_month=dorm_train.groupby('month')['month'].count()
a=['01','08','02']
def get_month_transform(month):
    if month in a:
        return 'v'
    else:
        return 'o'
    
dorm_test['month']=dorm_test['month'].apply(get_month_transform)

#对month进行one-hot编码
month_dummies=pd.get_dummies(dorm_test['month'])
month_dummies.columns=['dorm_on_school','dorm_vacation']
dorm_test=dorm_test.join(month_dummies)

In [13]:
#dorm_test.to_csv('dorm_input/dorm_test.csv',index=False)

In [14]:
dorm_test.to_csv('dorm_input/dorm_test_2.csv',index=False)

In [15]:
dorm_test.head()

,id,dorm_date,out_or_in,date,year,month,hour,weekday,dorm_hour00,dorm_hour01,...,dorm_hour23,dorm_hour_01_05,dorm_hour_morning,dorm_hour_afternoon,dorm_hour_evening,dorm_hour_midnight,dorm_resting_day,dorm_working_day,dorm_on_school,dorm_vacation
0,19326,2014-01-21 06:12:13,0,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
1,15190,2014-01-21 06:49:12,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
2,18610,2014-01-21 06:51:00,0,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
3,462,2014-01-21 07:40:51,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
4,18610,2014-01-21 07:51:06,1,01-21-2014,2014,v,b,wd,0,0,...,0,0,1,0,0,0,0,1,0,1
